In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFaceHub
from langchain_core.embeddings import Embeddings
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.retrievers import BM25Retriever
import numpy as np
import requests
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter("ignore")

In [2]:
def read_text_files(path: str) -> list[str]:
    # txtファイルを読み出す関数
    with open(path, "r") as f:
        contents = f.readlines() 
    return contents
api_key = read_text_files("api.txt")[0]

In [3]:
# モデルとtokenizerのダウンロード
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it", token = api_key)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", token = api_key)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### tokenizer練習

In [4]:
# 文字列の数値化
encoded_string = tokenizer.encode("I love apples.<eos>")
print(encoded_string)

[2, 235285, 2182, 34188, 235265, 1]


In [5]:
# 数値化された文字列の復元
decoded_string = tokenizer.decode(encoded_string)
print(decoded_string)

<bos>I love apples.<eos>


### 文字入力-> encode -> 文字出力 -> decode実演

In [6]:
#計算を早くしたい場合はGPUの使用をおすすめします。
encoded_string = tokenizer.encode("I love apples.", return_tensors= "pt") #encode
output = model.generate(encoded_string, max_length = 128) #文字列をencode数値データから文章を表す新たな文字列を生成。
print(f"モデルの出力: {output}")
decoded_string = tokenizer.decode(output[0].tolist()) #decode
print(f"出力の文字列: {decoded_string}")

モデルの出力: tensor([[     2, 235285,   2182,  34188, 235265,    590,   2182,    573,   1703,
            984,   1612, 235269,    573,   1703,    984,  10122, 235269,    578,
            573,   1703,    984,   1501,    682,   2375, 235265,    109,   1841,
            708,   1009,   5742,    577,   4313,  34188, 235336,    109,    688,
          60614,  66058,    109, 235287,  92749,  34188,    675,  46602,    578,
         125548,    108, 235287,  42511, 235290, 235266,  22875,  34188,    675,
          25470,    578,  35812,    108, 235287,  97566,  91370,    108, 235287,
          97566,   4506,    109,    688,  91469,   6422,  66058,    109, 235287,
          62655,  34188,   7528,    108, 235287,  47228,  34188,    577,  80815,
            578,  59105,    108, 235287,  25302,  15491,  75395,    108, 235287,
          62655,  34188,    675,  54154,  10605,    689,  13624,    109,    688,
           6273,  66058,    109, 235287,   9865,  75395,  41930,    108, 235287,
           9865,  17

### LangChain互換モデルへ変換

In [7]:
# HuggingFaceHubでLangChain互換のモデルをHuggingFaceからモデルをダウンロードするように作ることができる。
llm = HuggingFaceHub(repo_id="google/gemma-2b-it", model_kwargs={"temperature":0.5, "max_length":8192, "topk": 40}, huggingfacehub_api_token = api_key)

In [8]:
# RAGなしで大谷翔平について聞いてみる
response = llm.predict("大谷翔平って誰")
print(response)

大谷翔平って誰の？

大谷翔平は、日本の政治家・政治学者。自由民主党所属。

彼は、日本の政治における重要な役割を果たしている人物として有名です。


In [9]:
# 野球テキストデータを定義
baseball_db = read_text_files("baseball_db.txt")
# Embeddingを行うモデルを定義、今回はHuggingFaceHubにある備え付けのEmbeddingsを使う
embedder = HuggingFaceHubEmbeddings(huggingfacehub_api_token=api_key)


In [10]:
# promptに大谷翔平の情報を加える、RAGの例を行う。
prompt = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: 
""".format(context = baseball_db[0], question = "大谷翔平って誰, どこのチーム?")
print(prompt)

Answer the question based only on the following context:
大谷 翔平（おおたに しょうへい、1994年7月5日 - ）は、岩手県水沢市（現：奥州市）出身のプロ野球選手（投手、指名打者、外野手）。右投左打。MLBのロサンゼルス・ドジャース所属。

Question: 大谷翔平って誰, どこのチーム?
Answer: 



In [11]:
response = llm.predict(prompt)
print(response)

Answer the question based only on the following context:
大谷 翔平（おおたに しょうへい、1994年7月5日 - ）は、岩手県水沢市（現：奥州市）出身のプロ野球選手（投手、指名打者、外野手）。右投左打。MLBのロサンゼルス・ドジャース所属。

Question: 大谷翔平って誰, どこのチーム?
Answer: 
大谷翔平は、岩手県水沢市出身のプロ野球選手（投手、指名打者、外野手）。MLBのロサンゼルス・ドジャース所属。


### Embedding体験

In [12]:
# Embeddingを得る関数
def get_embedding(sentence: str,embedder: Embeddings = HuggingFaceHubEmbeddings()) -> np.array:
    embedding = embedder.embed_query(sentence)
    return np.array(embedding)
# 2つの単語の間の類似度を計算する
def cosine_similarity_with_two_words(sentence_1: str, sentence_2: str, embedder: Embeddings = HuggingFaceHubEmbeddings()) -> float:
    embedding_1 = get_embedding(sentence_1, embedder)
    embedding_2 = get_embedding(sentence_2, embedder)
    cosine_similarity = embedding_1 @ embedding_2 / np.linalg.norm(embedding_1) / np.linalg.norm(embedding_2)
    return cosine_similarity

In [13]:
print(cosine_similarity_with_two_words("回転焼き", "大判焼き", embedder))

# 大判焼きと回転焼きは非常によく似ていることがわかる。
#他の単語でもやってみましょう。もし似ているはずの単語なのに類似度が低い場合、学習時のコーパスに無いなどの問題が起こっている可能性があります。

0.8916292500727923


### RAG用ベクトルデータベース構築

In [14]:
baseball_db = read_text_files("baseball_db.txt")

In [15]:
# langchainのFAISSのベクトルデータストアを定義
vectorstores = FAISS.from_texts(baseball_db, embedder)

In [16]:
# vectorstoresをlangchainのretrieverとして変換、search_kwargs = {"k": 1}は類似度が上位1位のものを選ぶことを意味する
retriever = vectorstores.as_retriever(search_kwargs = {"k": 1})

In [17]:
#RAGの定義
rag_llm = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents = True, # ここをTrueにすることで回答に使った文章を出すことができる
)

In [18]:
response = rag_llm.invoke("大谷翔平って誰ですか")
print(response["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

大谷 翔平（おおたに しょうへい、1994年7月5日 - ）は、岩手県水沢市（現：奥州市）出身のプロ野球選手（投手、指名打者、外野手）。右投左打。MLBのロサンゼルス・ドジャース所属。


Question: 大谷翔平って誰ですか
Helpful Answer: 大谷翔平は、岩手県水沢市の出身のプロ野球選手です。


In [19]:
# ドキュメントの出力
for doc in response["source_documents"]:
    print(doc.page_content)

大谷 翔平（おおたに しょうへい、1994年7月5日 - ）は、岩手県水沢市（現：奥州市）出身のプロ野球選手（投手、指名打者、外野手）。右投左打。MLBのロサンゼルス・ドジャース所属。



### Linux RAGを作る

In [20]:
# requests ライブラリとBeautifulSoupを用いてLinuxの情報を含んだテキストデータベースを制作する。
# <p>タグに多く情報があるのでそれを用いる
url = "https://ja.wikipedia.org/w/api.php"
params = {
    "action" : "parse",
    "page" : "Linux",
    "format" : "json",
    "prop" : "text",
}
response = requests.get(url=url, params=params)
content: dict = response.json()
html_content = content["parse"]["text"]["*"]
html = BeautifulSoup(html_content)
articles = []
for article in html.find_all("p"):
    articles.append(article.text)

In [21]:
response = llm.predict("Linuxのペンギンのあの名前は?")
print(response)

Linuxのペンギンのあの名前は?

ペンギンの名前は、ペンギンのような形をした物体や、ペンギンのような動作をするものに対する名前の由来と考えられます。

ペンギンの名前は、日本語では「ペンギンの名前」や「ペンギンのようなもの」と翻訳されます。

ペンギンの名前は、日本語では「ペンギンの名前」と翻訳されます。


In [22]:
prompt = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: 
""".format(context = articles[-1],question = "Linuxのあのペンギンの名前は?")

In [23]:
response = llm.predict(prompt)
print(response)

Answer the question based only on the following context:
LinuxカーネルVersion 2.x系列登場後のマスコットには、リーナス・トーバルズの嗜好を汲んで、タックス (Tux) と名付けられたペンギンのキャラクターが選ばれている。
また、Linuxカーネル Version 2.6.29限定のマスコットとして、タスマニアデビルのTuzが発表[108]されている。

Question: Linuxのあのペンギンの名前は?
Answer: 
タックス (Tux)


In [24]:
NUMBER_OF_RETRIEVERS = 4
retriever = FAISS.from_texts(articles, embedder).as_retriever(search_kwargs = {"k": NUMBER_OF_RETRIEVERS})

In [25]:
rag_llm = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents = True, # ここをTrueにすることで回答に使った文章を出すことができる
)

In [26]:
response = rag_llm.invoke("Linuxのあのペンギンの名前は?")
print(response["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Linux（リナックス、他の読みは#「Linux」の読み方で後述）とは、狭義にはUnix系オペレーティングシステムカーネルであるLinuxカーネルを指し、広義にはそれをカーネルとして周辺を整備したシステム全体のことをいう（GNU/Linuxも参照）。


Linuxが動作している携帯電話やPDAは2007年頃から一般的になりだした。例えば、ノキア N810、オープンモコ社のNeo1973、モトローラ社のROKR E3などがある。このトレンドは続き、パーム社はLinuxベースのwebOSを開発した。これはPalm Preスマートフォンに使用された。


Linuxは、狭義にはLinuxカーネル、広義にはそれをカーネルとして用いたオペレーティングシステムを指す。


他の有名な現代的OSとの主要な違いとして、Linuxカーネルおよびその他の構成要素がフリーかつオープンソースであることが挙げられる。そのようなOSはLinuxだけではないが、Linuxはその中でも突出して広く使われている[23]。


Question: Linuxのあのペンギンの名前は?
Helpful Answer: The context does not provide the name of the pengin, so I cannot answer this question from the context.


In [27]:
response = rag_llm.invoke("LinuxカーネルVersion 2.x系列登場後のマスコットの名前は?")
print(response["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

LinuxカーネルVersion 2.x系列登場後のマスコットには、リーナス・トーバルズの嗜好を汲んで、タックス (Tux) と名付けられたペンギンのキャラクターが選ばれている。
また、Linuxカーネル Version 2.6.29限定のマスコットとして、タスマニアデビルのTuzが発表[108]されている。


Linuxのソースコードは肥大化を続ける傾向にあり、これを防ぐために古いコードやマイナーなデバイスドライバ用のコードを削除する活動が行われている。


最初のLinuxのリリースまでの開発はおよそ4カ月かけて行われた。


2008年には東京証券取引所の基幹システムのひとつ「派生売買システム」でLinuxが使われるようになった。 以降、東証ではシステムのLinux化が進められている。


Question: LinuxカーネルVersion 2.x系列登場後のマスコットの名前は?
Helpful Answer: タックス (Tux)

The context does not specify the name of the mascot for Linux kernel version 2.x series, so I cannot answer this question from the provided context.


### BM25の場合
ベクトル検索の他に、ベクトル検索を用いずにBM25を用いた場合の性能を見る。

In [28]:
# Defaultのlangchain BM25は日本語に対応していないため少し工夫をする
retriever = BM25Retriever.from_texts(articles, preprocess_func=tokenizer.tokenize) #preprocess_funcでどういう処理を行うかを指定する。gemmmaのtokenizerのtokenizeメソッドを指定してやれば良い。

In [29]:
retriever.get_relevant_documents("Linux マスコット")

[Document(page_content='LinuxカーネルVersion 2.x系列登場後のマスコットには、リーナス・トーバルズの嗜好を汲んで、タックス (Tux) と名付けられたペンギンのキャラクターが選ばれている。\nまた、Linuxカーネル Version 2.6.29限定のマスコットとして、タスマニアデビルのTuzが発表[108]されている。\n'),
 Document(page_content='今日ではLinuxの普及に伴い国際規格が策定されている。Linuxカーネルを使用し、Linux Standard Base (LSB) Core Specification (ISO/IEC 23360シリーズ) に準拠したOSが、OSとしてのLinuxであるとされている。\n'),
 Document(page_content='デスクトップやサーバ用のLinuxは、Linuxディストリビューションという形でパッケージ化されて配布されている。有名なLinuxディストリビューションとしては、Debian（とその派生であるUbuntu、Linux Mint）、Red Hat Linux（とその派生であるFedora、Red Hat Enterprise Linux、CentOS）、Mandriva Linux/Mageia、openSUSE、Arch Linuxなどがある。各Linuxディストリビューションは、Linuxカーネル、システムソフトウェア、ライブラリ等、巨大なコンパイル済のアプリケーション群を含んでいる。\n'),
 Document(page_content='Linux（リナックス、他の読みは#「Linux」の読み方で後述）とは、狭義にはUnix系オペレーティングシステムカーネルであるLinuxカーネルを指し、広義にはそれをカーネルとして周辺を整備したシステム全体のことをいう（GNU/Linuxも参照）。\n')]

In [30]:
rag_llm = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents = True, # ここをTrueにすることで回答に使った文章を出すことができる
)

In [31]:
response = rag_llm.invoke("Linuxのあのペンギンの名前は?")
print(response["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

LinuxカーネルVersion 2.x系列登場後のマスコットには、リーナス・トーバルズの嗜好を汲んで、タックス (Tux) と名付けられたペンギンのキャラクターが選ばれている。
また、Linuxカーネル Version 2.6.29限定のマスコットとして、タスマニアデビルのTuzが発表[108]されている。


1991年の9月、開発を促進するために、Linuxのファイルはヘルシンキ工科大学のFTPサーバ (ftp.funet.fi) にアップロードされた。トーバルズの協力者であり、当時そのサーバの責任者であったレムケは、「Freax」という名前を良く思わず（「Freax」と語感が酷似している「Freaks」は英語で変人・奇人の意味を持つため）、彼はトーバルズに相談することなく、サーバ上のプロジェクトに勝手に「Linux」という名前をつけてしまった。その後トーバルズも、その名前に同意した。


IDC社の2007年第1四半期の調査は、その時点でLinuxは全サーバの12.7%を占めていると示した[47]。ただしこの数値は、様々な企業によってLinuxサーバとして販売されたマシンの台数だけに基づく推定であり、サーバハードウェアを購入したあとでLinuxをインストールしたものを考慮に入れていない。2008年9月には、マイクロソフト社のCEOスティーブバルマーが、ウェブサーバの60%はLinuxシステムであり、それに対してWindows Serverは40%であることを認めた[48]。


米国では、「Linux」という名前はリーナス・トーバルズが登録している商標である。初期は誰もこの名前を登録していなかったが、1994年8月15日に William R. Della croce, Jrが出願を行い、Linuxディストリビュータ達にロイヤリティを要求するということが起きた。1996年にリーナス・トーバルズといくつかの団体が、商標をリーナス・トーバル

In [32]:
response = rag_llm.invoke("Linuxのライセンスは?")
print(response["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

LinuxおよびほとんどのGNUソフトウェアは、ライセンスとして GNU General Public License (GPL) を採用している。GPLでライセンスされていることにより、Linuxを再頒布する者はソースコード（加えた修正も含む）を同じ条項で入手可能にすることが要求される。他の主要コンポーネントの中には別のライセンスを使っているものもある。例えば、多くのライブラリはGNU Lesser General Public License (LGPL)（GPLよりも許諾的）を採用しており、X.orgはMITライセンスを採用している。


フリーかつオープンソースなライセンスの一部は、コピーレフトという原理に基づいている。コピーレフトはある種の相互関係と捉えられる。コピーレフトなライセンスで公開されているソフトウェアのソースコード片は自由に利用できるが、それを利用して作ったソフトウェアを一般に頒布する場合はそれ自身もコピーレフトなライセンスでソースコードを公開しなければならない。最も一般的なフリーソフトウェアライセンスのひとつである「GNU 一般公衆利用許諾書」(GNU GPL) はコピーレフトの一形態であり、LinuxカーネルやGNUプロジェクトの多くのコンポーネントのライセンスとして採用されている。


Linuxディストリビューションに含まれるソフトウェアパッケージの多くはフリーソフトウェアライセンスを採用している。フリーソフトウェアライセンスは、商業利用を明示的に許諾しており、さらにはそれを推奨している。多くのLinuxディストリビューションは無償で入手できるが、いくつかの大企業は商用版ディストリビューションを販売することで利益を得ている。これらのディストリビューションでは、（特にビジネスユーザ向けの）サポートサービスが提供されており、さらに、プロプライエタリなサポートパッケージや、大量のインストールを行ったり管理作業を簡略化する